In [148]:
import pandas as pd
import os


def load_and_combine_pkl_files(directory_path):
    # List to hold all the dataframes
    dataframes = []

    # Loop through all the files in the specified directory
    for filename in os.listdir(directory_path):
        if filename.endswith("45100_45300_new_gradient_top50_new_perturbed_inferenced_df.pkl"):
            # Construct full file path
            file_path = os.path.join(directory_path, filename)
            # Load the dataframe from a pkl file
            df = pd.read_pickle(file_path)
            # Append the dataframe to the list
            dataframes.append(df)

    # Concatenate all dataframes into one big dataframe
    big_df = pd.concat(dataframes, ignore_index=True)

    return big_df


# Usage
directory_path = '/Users/ximing/Desktop/Explainprompt/top_k'
big_df = load_and_combine_pkl_files(directory_path)
big_df = big_df



In [131]:
print(big_df.columns)

Index(['prompt', 'real_output', 'token_level', 'word_level', 'label',
       'component_level', 'instruction', 'query', 'component_range',
       'instruction_weight', 'query_weight', 'exec_time', 'gpu_memory_usage',
       'instructions_tokens', 'query_tokens', 'max_normalized_value',
       'max_token', 'instruction_token_top_0.25_peturbed',
       'instruction_token_bottom_0.25_peturbed',
       'query_token_top_0.25_peturbed', 'query_token_bottom_0.25_peturbed',
       'top_reconstructed_instruction_0.25', 'top_reconstructed_query_0.25',
       'bottom_reconstructed_instruction_0.25',
       'bottom_reconstructed_query_0.25',
       'top_reconstructed_instruction_0.25_result',
       'top_reconstructed_query_0.25_result',
       'bottom_reconstructed_instruction_0.25_result',
       'bottom_reconstructed_query_0.25_result'],
      dtype='object')


In [143]:
big_df['sentiment'] = big_df['real_output'].apply(lambda x: 1 if 'POS' in x else (0 if 'NEG' in x else -1))
big_df['sentiment_top_reconstructed_instruction'] = big_df['top_reconstructed_instruction_0.25_result'].apply(lambda x: 1 if 'POS' in x else (0 if 'NEG' in x else -1))
big_df['sentiment_top_reconstructed_query'] = big_df['top_reconstructed_query_0.25_result'].apply(lambda x: 1 if 'POS' in x else (0 if 'NEG' in x else -1))
big_df['sentiment_bottom_reconstructed_instruction'] = big_df['bottom_reconstructed_instruction_0.25_result'].apply(lambda x: 1 if 'POSISTIVE' in x else (0 if 'NEGATIVE' in x else -1))
big_df['sentiment_bottom_reconstructed_query'] = big_df['bottom_reconstructed_query_0.25_result'].apply(lambda x: 1 if 'POS' in x else (0 if 'NEG' in x else -1))


In [135]:
differences = big_df['sentiment'] != big_df['sentiment_bottom_reconstructed_instruction']
print(differences.sum() / len(differences))

0.5757575757575758


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython_darwin_312_64.pyx", line 1527, in _pydevd_bundle.pydevd_cython_darwin_312_64.ThreadTracer.__call__
  File "/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/pydev/_pydevd_bundle/pydevd_kill_all_pydevd_threads.py", line 5, in kill_all_pydev_threads
    def kill_all_pydev_threads():
    
KeyboardInterrupt


In [134]:
print(big_df['sentiment_bottom_reconstructed_query'])

0      0
1      0
2      0
3      0
4      0
      ..
193    0
194    0
195    0
196    1
197    0
Name: sentiment_bottom_reconstructed_query, Length: 198, dtype: int64


In [144]:
differences = big_df['sentiment'] != big_df['sentiment_top_reconstructed_instruction']
print(differences.sum() / len(differences))

0.898989898989899


In [145]:
differences = big_df['sentiment'] != big_df['sentiment_top_reconstructed_query']
print(differences.sum() / len(differences))

0.2777777777777778


In [146]:
differences = big_df['sentiment'] != big_df['sentiment_bottom_reconstructed_instruction']
print(differences.sum() / len(differences))

0.6616161616161617


In [147]:
differences = big_df['sentiment'] != big_df['sentiment_bottom_reconstructed_query']
print(differences.sum() / len(differences))

0.22727272727272727


In [76]:
expanded_df = pd.DataFrame([word for sublist in big_df['instructions_tokens'] for word in sublist])


In [31]:
grouped_stats = expanded_df.groupby('token').agg({
    'value': ['sum', 'mean']  # 计算总和和均值
}).reset_index()
grouped_stats.columns = ['token', 'total_value', 'average_value']  # 重新命名列以便阅读
print(grouped_stats)

     token  total_value  average_value
0  Respond     7.035245       0.071788
1        a     6.954704       0.070966
2     form     6.981214       0.071237
3       in     6.979714       0.071222
4     long     6.955979       0.070979
5       of     6.989855       0.071325
6    story     6.931501       0.070730
7      the     6.994080       0.071368


In [120]:
 print(big_df['bottom_reconstructed_instruction_0.25_result'])

0      POSITIVE^.\n\nThe band's ability to create a c...
1      INSUFFICIENT^ . I think it 's important to rec...
2      ated and does n't make sense. I apologize, but...
3      NEGATIVE^.\n\nAnswer:\nThe sentiment of the pr...
4      SARCASTIC^.\n\n^POSITIVE^ Oh, wow, what a fasc...
                             ...                        
193    POSITIVE^.\n\nIn Auteil's less dramatic but eq...
194    NEUTRAL^.\n\nAnswer:\n\nThe sentiment of the p...
195    NEGATIVE^.\n\nThe story is a thought-provoking...
196    NEGATIVE^.\n\nThe historical fiction novel, "T...
197    POSITIVE^ because the sentence emphasizes the ...
Name: bottom_reconstructed_instruction_0.25_result, Length: 198, dtype: object


discretize 

In [27]:
grouped_stats = expanded_df.groupby('token').agg({
    'value': ['sum', 'mean']  # 计算总和和均值
}).reset_index()
grouped_stats.columns = ['token', 'total_value', 'average_value']  # 重新命名列以便阅读
print(grouped_stats)

     token  total_value  average_value
0  Respond     6.650957       0.067867
1        a     6.684485       0.068209
2     form     6.512113       0.066450
3       in     6.794886       0.069336
4     long     7.163081       0.073093
5       of     6.620741       0.067559
6    story     7.846718       0.080069
7      the     6.574434       0.067086


simliarity 

In [21]:
grouped_stats = expanded_df.groupby('token').agg({
    'value': ['sum', 'mean']  # 计算总和和均值
}).reset_index()
grouped_stats.columns = ['token', 'total_value', 'average_value']  # 重新命名列以便阅读
print(grouped_stats)

     token  total_value  average_value
0  Respond     4.941783       0.050426
1        a     5.138980       0.052439
2     form     5.004630       0.051068
3       in     5.109278       0.052135
4     long     5.712436       0.058290
5       of     5.075337       0.051789
6    story     7.830939       0.079908
7      the     4.838620       0.049374


In [ ]:
print(big_df.iloc[4]['word_level'])

In [ ]:
print(big_df['bottom_reconstructed_query_0.2_result'][:20])


In [11]:
differences = big_df['real_output'] != big_df['bottom_reconstructed_query_0.2_result']
print(differences.sum()/len(differences))

0.07142857142857142


In [12]:
differences = big_df['real_output'] != big_df['top_reconstructed_query_0.2_result']
print(differences.sum()/len(differences))

0.35714285714285715


In [11]:
differences = big_df['real_output'] != big_df['bottom_reconstructed_instruction_0.2_result']
print(differences.sum()/len(differences))

0.030612244897959183


In [14]:
differences = big_df['real_output'] != big_df['top_reconstructed_instruction_0.2_result']
print(differences.sum()/len(differences))

1.0


In [15]:
print(big_df['instruction_weight'].mean())

0.8787098084681545


In [16]:
print(big_df['query_weight'].mean())

0.12129019153184557


In [12]:
print(big_df['exec_time'].mean())

17.86105881661785


In [13]:
print(big_df['input_cost'].mean())

1426.1836734693877


In [14]:
print(big_df['output_cost'].mean())


154.48979591836735


In [15]:
print(big_df['total_cost'].mean())


1580.6734693877552


In [36]:
import numpy as np

In [37]:
x = np.array([[7.1, 7.1, 7.2], [8.3, 9.4, 10.5]])

In [38]:
from scipy import stats

In [39]:
res = stats.spearmanr(x)

In [43]:
x = np.array([7.1, 7.1, 7.2])
y = np.array([8.3, 9.4, 10.5])
res = stats.spearmanr(x,y)

print(res)

SignificanceResult(statistic=0.8660254037844387, pvalue=0.3333333333333332)


In [48]:
import numpy as np
from scipy.stats import spearmanr

# Create a 2D array with sample data
# Let's assume each row is an observation and each column is a different variable
# data = np.array([
#     [1, 2, 3.5],
#     [2, 3, 2.5],
# 
# ])
data = np.array([
    [1, 2, 3, 4, 5],     # Variable 1
    [2, 3, 2, 1, 0],     # Variable 2
    [3.5, 2.5, 0.5, 1.5, 3.0]  # Variable 3
]).T
# Calculate the Spearman correlation coefficient and p-value
correlation, p_value = spearmanr(data)

print("Spearman correlation coefficient matrix:\n", correlation)
print("P-value matrix:\n", p_value)


Spearman correlation coefficient matrix:
 [[ 1.         -0.82078268 -0.3       ]
 [-0.82078268  1.         -0.10259784]
 [-0.3        -0.10259784  1.        ]]
P-value matrix:
 [[1.40426542e-24 8.85870053e-02 6.23837665e-01]
 [8.85870053e-02 0.00000000e+00 8.69597921e-01]
 [6.23837665e-01 8.69597921e-01 1.40426542e-24]]
